# Inits

In [1]:
from energy_management_client import BackendPythonClient
import pandas as pd
import json

In [2]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

# List of Building IDs

In [3]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID (from customer object), address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias, gfid,
      - LayoutID (extracted from modular_system)

    Usage:
        customers = client.customers.get_list_of_customers()
        df = get_buildings_df(client, customers)
        print(df.head())
    """
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            
            # Get buildings
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id         = getattr(b, 'building_id', None)
                address             = getattr(b, 'address', None)
                city                = getattr(b, 'city', None)
                postal_code         = getattr(b, 'postal_code', None)
                coordinates         = getattr(b, 'coordinates', None)
                CustomerAlias       = getattr(b, 'CustomerAlias', None)
                gfid                = getattr(b, 'gfid', None)
                
                # Parse modular_system
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customer_id,  # <-- corrected here
                    "customer_name": customer_name,
                    "address": address,
                    "city": city,
                    "postal_code": postal_code,
                    "gfid": gfid,
                })

        return pd.DataFrame(data)

    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

In [4]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df_buildings = get_buildings_df(client, customers)

In [5]:
#test
df_buildings.tail()

,building_id,customerID,customer_name,address,city,postal_code,gfid
2195,2504,191,Wohnungs-Genossenschaft,Weinsbergstr. 170,Köln,50823,OPS-23523
2196,2505,191,Wohnungs-Genossenschaft,Odenwaldstr. 274,Köln,50999,OPS-23530
2197,2531,193,WoBaG Bamberg,Reußstraße 25,Bamberg,96052,OPS-23836
2198,2532,193,WoBaG Bamberg,Weißenburgstraße 19,Bamberg,96052,OPS-23837
2199,2533,193,WoBaG Bamberg,Theodor-Heuss-Ring 9,Bamberg,96050,OPS-23838


# Get Device

In [13]:
def get_devices(client):
    """
    usage:
    list_device=get_devices(client)
    list(list_device)
    """
    devices_list=client.devices.get_devices()
    devices_list=list(devices_list)[0][1]
    device_dicts = [device.__dict__ for device in devices_list]
    df_device = pd.DataFrame(device_dicts)
    return df_device

In [14]:
df_devices=get_devices(client)
df_devices = df_devices.rename(columns={"type": "device_type"})

In [18]:
df_buildings = df_buildings.merge(
    df_devices[['building_id', 'device_type', 'device_id']],
    on='building_id',
    how='left'  # ensures all rows from df_buildings are kept
)

# Get sub components

In [6]:
def get_sub_component(building_id):
    subsystems = client.buildings.get_sub_systems_by_building_id(building_id)
    
    subsystems_infos = {
        f"{s.acronym}({s.hydraulic_location_index})": f"category : {s.category} type : {s.type} sub_type : {s.sub_type}"
        for s in subsystems
    }
    return subsystems_infos

In [7]:
client.buildings.get_sub_systems_by_building_id(2382) 


[BuildingSubSystem(id=220622, acronym='global', hydraulic_location_index=1, building_id=2382, type='GLOBAL', sub_type='NOT_SPECIFIED', category='NOT_SPECIFIED', remote_status='NOT_SPECIFIED', controller_id=None),
 BuildingSubSystem(id=220623, acronym='dh', hydraulic_location_index=1, building_id=2382, type='DISTRICT_HEATING', sub_type='NOT_SPECIFIED', category='PRODUCER', remote_status='NOT_SPECIFIED', controller_id=None),
 BuildingSubSystem(id=220624, acronym='hc(n)', hydraulic_location_index=1, building_id=2382, type='HEATING_CIRCUIT', sub_type='FLOOR_HEATING', category='CONSUMER', remote_status='NOT_SPECIFIED', controller_id=None),
 BuildingSubSystem(id=220625, acronym='hw(n)', hydraulic_location_index=1, building_id=2382, type='WARM_WATER', sub_type='FRESH_WATER_STATION', category='CONSUMER', remote_status='NOT_SPECIFIED', controller_id=None)]

In [8]:
get_sub_component(2382)

{'global(1)': 'category : NOT_SPECIFIED type : GLOBAL sub_type : NOT_SPECIFIED',
 'dh(1)': 'category : PRODUCER type : DISTRICT_HEATING sub_type : NOT_SPECIFIED',
 'hc(n)(1)': 'category : CONSUMER type : HEATING_CIRCUIT sub_type : FLOOR_HEATING',
 'hw(n)(1)': 'category : CONSUMER type : WARM_WATER sub_type : FRESH_WATER_STATION'}

In [9]:
df_buildings['sub_components'] = df_buildings['building_id'].apply(get_sub_component)

In [10]:
df_buildings.tail()

,building_id,customerID,customer_name,address,city,postal_code,gfid,sub_components
2195,2504,191,Wohnungs-Genossenschaft,Weinsbergstr. 170,Köln,50823,OPS-23523,{}
2196,2505,191,Wohnungs-Genossenschaft,Odenwaldstr. 274,Köln,50999,OPS-23530,{}
2197,2531,193,WoBaG Bamberg,Reußstraße 25,Bamberg,96052,OPS-23836,{}
2198,2532,193,WoBaG Bamberg,Weißenburgstraße 19,Bamberg,96052,OPS-23837,{}
2199,2533,193,WoBaG Bamberg,Theodor-Heuss-Ring 9,Bamberg,96050,OPS-23838,{}


# Get building sensors

In [11]:
client.buildings.get_list_of_sensors_by_building_id(2382)

[BuildingSensorDto(id=36491, gf_standard_id=460, sub_system_id=220622, hydraulic_location_index=1, factor=1, current_value=None, node_id='YC', acronym='e_g_bldg_y', unit='kWh', long_name='Energie Gebäude Global Jährlich', short_name='E BLDG GL ', source='MANUALLY_ENTERED'),
 BuildingSensorDto(id=40030, gf_standard_id=88, sub_system_id=220623, hydraulic_location_index=1, factor=0.1, current_value=718, node_id='1', acronym='t_dh_primflow', unit='°C', long_name='Vorlauftemperatur Primär Fernwärme ', short_name='T VLpri FW', source='GREENBOX_MQTT'),
 BuildingSensorDto(id=40031, gf_standard_id=None, sub_system_id=None, hydraulic_location_index=1, factor=1, current_value=0, node_id='2', acronym=None, unit=None, long_name=None, short_name=None, source='GREENBOX_MQTT'),
 BuildingSensorDto(id=40032, gf_standard_id=94, sub_system_id=220623, hydraulic_location_index=1, factor=0.1, current_value=700, node_id='3', acronym='t_dh_primreturn', unit='°C', long_name='Rücklauftemperatur Primär Fernwärme'

In [17]:
df_merged.columns

Index(['building_id', 'customerID', 'customer_name', 'address', 'city',
       'postal_code', 'gfid', 'sub_components', 'device_type', 'device_id'],
      dtype='object')